In [1]:
!pip install Pillow

In [2]:
#Importing OpenCV Library for basic image processing functions
import cv2
# Numpy for array related functions
import numpy as np
# Dlib for deep learning based Modules and face landmark detection
import dlib
#face_utils for basic operations of conversion
from imutils import face_utils
#winsound for beep sound
import winsound
import tkinter as tk
from PIL import Image, ImageTk

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
# Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
#status marking for current state
# Variables to track eye blink behavior
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [6]:
def compute(ptA,ptB):
    # find the euclidean 
	dist = np.linalg.norm(ptA - ptB)
	return dist

In [7]:
def blinked(a,b,c,d,e,f):
	up = compute(b,d) + compute(c,e)
	down = compute(a,f)
	ratio = up/(2.0*down)

	#Checking if it is blinked
	if(ratio>0.25):
		return 2
	elif(ratio>0.21 and ratio<=0.25):
		return 1
	else:
		return 0

In [8]:
# Global variables for drowsiness detection
cap = None
label = None
sleep = 0
drowsy = 0
active = 0
status = "Active"
color = (0, 255, 0)

# Function to perform drowsiness detection
def detect_drowsiness():
    global cap, label, sleep, drowsy, active, status, color 

    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame is valid
    if not ret:
        print("Error: Failed to capture frame.")
        return

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for enhancing image contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhancedImage = clahe.apply(gray)

    # Detect faces in the grayscale frame (Assuming 'detector' and 'predictor' are initialized)
    faces = detector(enhancedImage)

    # Process each detected face
    for face in faces:
        # Extract the coordinates of the bounding box around each face
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()

        # Draw a green rectangle around the face
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Detect facial landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Calculate eye blinks
        left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        # Determine eye blink behavior and update status accordingly
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING"
                color = (255, 0, 0)
                # alert sound or notification here (e.g., winsound.Beep(1000, 500))
                winsound.Beep(1000, 500)
        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active"
                color = (0, 255, 0)

        # Calculate the position to put the status text
        text_x = x1
        text_y = y1 - 10  # Adjust this value to place the text above the face rectangle

        # Display eye blink status (Awake, Drowsy, or Sleeping)
        cv2.putText(frame, status, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

    # Convert the OpenCV frame to RGB format for tkinter display
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  #converts captured frame to RGB formate
    img = Image.fromarray(frame_rgb)  #create a PIL(Python Imaging library) image object
    img_tk = ImageTk.PhotoImage(image=img)  #converts PIL image to tkinter compatible formate

    # Update the label widget with the new image
    label.configure(image=img_tk)
    label.image = img_tk  # Keep a reference to prevent garbage collection

    # Call this function again after a delay (10 milliseconds) for real-time processing
    if cap.isOpened():  # Check if the capture device is still open
        label.after(10, detect_drowsiness)
    else:
        print("Webcam capture device closed.")

# Function to start drowsiness detection
def start_detection():
    global cap, label

    # Initialize video capture object 'cap' (device index 0)
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened successfully
    if not cap.isOpened():
        print("Error: Failed to open webcam.")
        return

    # Create a new tkinter window for displaying the webcam feed over the base window
    root = tk.Toplevel()
    root.title("Drowsiness Detection")

    # Create a label widget to display the webcam feed
    label = tk.Label(root)
    tk.Frame(root_main, bg="black")
    label.pack(padx=10, pady=10)

    # Start detecting drowsiness
    detect_drowsiness()

    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     # Release the video capture and close all OpenCV windows
    #     cap.release()
    #     cv2.destroyAllWindows()
    #     return

    
    # Close the video capture and destroy the tkinter window when closed
    def on_closing():
        if cap.isOpened():
            cap.release()  # Release the webcam
        root.destroy()  # Destroy the tkinter window

    root.protocol("WM_DELETE_WINDOW", on_closing)


In [10]:
# Create the main tkinter window
root_main = tk.Tk()
root_main.title("Drowsiness Detection App")

# Calculate screen width and height
screen_width = root_main.winfo_screenwidth()
screen_height = root_main.winfo_screenheight()

# Set the size of the main window
window_width = 1400
window_height = 700
x_position = (screen_width - window_width) // 2
y_position = (screen_height - window_height) // 2

root_main.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")


# Create a label for the text "Drowsiness Detection"
label = tk.Label(root_main, text="Drowsiness Detection", fg="black", font=("Helvetica", 35))
label.pack(pady=40)  # Place the label with some padding at the top

# Create a start button to initiate drowsiness detection
start_button = tk.Button(root_main, text="Start", command=start_detection, bg="purple", fg="white", font=("arial", 20))
start_button.pack(pady=20, padx=50, ipadx=20, ipady=10, anchor=tk.CENTER)  # Center the button both horizontally and vertically

# Run the main tkinter event loop
root_main.mainloop()